### Minimal example of how to use FLAML for auto-causality
We use FLAML twice, first to find the best component model for each estimator, and then to find the best econML estimator. Here we show how to perform the first step

In [11]:
%load_ext autoreload
%autoreload 2
import os, sys

import pandas as pd
import numpy as np
import gzip
import dill
import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from collections import defaultdict


root_path = root_path = os.path.realpath('../..')
data_dir = os.path.realpath(os.path.join(root_path, "auto-causality/data"))
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)
print(data_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\Users\Timo\Desktop\osgd_wise\code\auto-causality\data


In [2]:
# run this cell if you are using dowhy and this repo by just pulling the source
sys.path.append(os.path.join(root_path, "dowhy"))
sys.path.append(os.path.join(root_path, "auto-causality"))


In [3]:
from dowhy import CausalModel

from auto_causality.utils import featurize, AutoMLWrapper
from auto_causality.params import SimpleParamService
from auto_causality.scoring import make_scores


C:\Users\Timo\anaconda3\envs\autocausality\lib\site-packages\statsmodels\compat\pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


### Control Parameters

In [5]:
# set all the control parameters here
train_size = 0.5
test_size = None
time_budget = 300
num_cores = os.cpu_count() - 1
conf_intervals = False


### Data Preprocessing

In [6]:
# load raw data
data = pd.read_csv(
    "https://raw.githubusercontent.com/AMLab-Amsterdam/CEVAE/master/datasets/IHDP/csv/ihdp_npci_1.csv",
    header=None,
)
col = [
    "treatment",
    "y_factual",
    "y_cfactual",
    "mu0",
    "mu1",
]
for i in range(1, 26):
    col.append("x" + str(i))
data.columns = col
# drop the columns we don't care about
ignore_patterns = ["y_cfactual", "mu"]
ignore_cols = [c for c in data.columns if any([s in c for s in ignore_patterns])]
data = data.drop(columns=ignore_cols)


# prepare the data

treatment = "treatment"
targets = ["y_factual"]  # it's good to allow multiple ones
features = [c for c in data.columns if c not in [treatment] + targets]

data[treatment] = data[treatment].astype(int)
# this is a trick to bypass some DoWhy/EconML bugs
data["random"] = np.random.randint(0, 2, size=len(data))

used_df = featurize(
    data, features=features, exclude_cols=[treatment] + targets, drop_first=False,
)
used_features = [
    c for c in used_df.columns if c not in ignore_cols + [treatment] + targets
]


# Let's treat all features as effect modifiers
features_X = [f for f in used_features if f != "random"]
features_W = [f for f in used_features if f not in features_X]


train_df, test_df = train_test_split(used_df, train_size=train_size)
if test_size is not None:
    test_df = test_df.sample(test_size)

test_df.to_csv(os.path.join(data_dir, f"test_{time_budget}.csv"))
train_df.to_csv(os.path.join(data_dir, f"train_{time_budget}.csv"))


### Model Parameters

In [27]:
# set the model parameters here
# component models:
propensity_model = DummyClassifier(strategy="prior")
outcome_model = AutoMLWrapper(
    fit_params={
        "time_budget": time_budget,
        "verbose": 1,
        "task": "regression",
        "n_jobs": num_cores,
        "pred_time_limit": 10 / 1e6,
    }
)

# instantiate model config:
cfg = SimpleParamService(
    propensity_model,
    outcome_model,
    conf_intervals=conf_intervals,
    n_bootstrap_samples=20,
    n_estimators=500,
    max_depth=10,
    min_leaf_size=2 * len(used_features),
)



### Model fitting & scoring
Here we fit a (selection of) model(s) to the data and score them with the ERUPT metric on held-out data

In [26]:
# get list of supported models

bulletpoint = "\n - "
print(f"supported models: {bulletpoint}{(bulletpoint).join(cfg.estimators())}")


supported models: 
 - backdoor.propensity_score_weighting
 - backdoor.econml.metalearners.SLearner
 - backdoor.econml.metalearners.TLearner
 - backdoor.econml.metalearners.XLearner
 - backdoor.econml.metalearners.DomainAdaptationLearner
 - backdoor.econml.dr.ForestDRLearner
 - backdoor.econml.dr.LinearDRLearner
 - backdoor.econml.dr.SparseLinearDRLearner
 - backdoor.econml.dml.LinearDML
 - backdoor.econml.dml.SparseLinearDML
 - backdoor.econml.dml.CausalForestDML
 - backdoor.auto_causality.dowhy_wrapper.direct_uplift.DirectUpliftDoWhyWrapper
 - backdoor.econml.orf.DROrthoForest
 - backdoor.econml.orf.DMLOrthoForest


1. Choose econML model and retrieve its default parameters

In [56]:
# 0. choose econML model and retrieve parameters
estimator = "backdoor.econml.dml.LinearDML"
estimator_params = cfg.method_params(estimator)

2. Instantiate the causal model with DoWhy

In [60]:
outcome = targets[0]
model = CausalModel(
    data=train_df,
    treatment=treatment,
    outcome=targets[0],
    common_causes=features_W,
    effect_modifiers=features_X,
)


3. Identify the effect with DoWhy

In [58]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)

4. Estimate the effect (this uses FLAML for the component models)

In [ ]:
estimates = model.estimate_effect(
    identified_estimand,
    method_name=estimator,
    control_value=0,
    treatment_value=1,
    target_units="ate",  # condition used for CATE
    confidence_intervals=conf_intervals,
    method_params=estimator_params,
)

5. Interpret the effect (i.e. how large is the treatment effect)

In [59]:
estimates.interpret()

Increasing the treatment variable(s) [treatment] from [0] to [1] causes an increase of 4.122409568331659 in the expected value of the outcome [y_factual], over the data distribution/population represented by the dataset.


6. Score the estimator (with ERUPT metric)

In [55]:
try:
    te_train = estimates.cate_estimates
    X_test = test_df[estimates.estimator._effect_modifier_names]
    te_test = estimates.estimator.estimator.effect(X_test).flatten()
    print(
        "manual mean impact:",
        te_train.mean(),
        te_test.mean(),
        "std:",
        te_train.std(),
    )
except:
    te_train = estimates.estimator.effect(train_df)
    te_test = estimates.estimator.effect(test_df)

scores = {
    "estimator": estimator,
    "outcome": outcome,
    "train": make_scores(model, train_df, te_train),
    "test": make_scores(model, test_df, te_test),
}

print(
    f"Scores for {estimator}_{outcome}",
    scores["train"]["erupt"],
    scores["test"]["erupt"],
)

manual mean impact: 4.122409568331659 4.515768093730235 std: 1.7655081756724051
Scores for backdoor.econml.dml.LinearDML_y_factual 6.4705822558812605 6.3390611739300935


Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.


In [13]:
# # add a baseline calculation
# train_df = pd.read_csv(os.path.join(data_dir, f"train_{time_budget}.csv"))
# test_df = pd.read_csv(os.path.join(data_dir, f"test_{time_budget}.csv"))
# # purge previous ones
# for outcome in targets:
#     def ate(df: pd.DataFrame):
#         return df[outcome][df[treatment]==1].mean() - df[outcome][df[treatment]==0].mean()

#     scores[outcome]["baseline"]={"estimator": "baseline",
#                                "outcome": outcome,
#                               "train":{"erupt": train_df[outcome].mean(),"ate": ate(train_df)},
#                               "test":{"erupt": test_df[outcome].mean(),"ate": ate(test_df)}}
                              
                              
